In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib
import geopandas as gp
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
import Outils

In [2]:
#importer les odnnées de  base
with ct.ConnexionBdd('local_otv') as c : 
    df = gp.read_postgis("select t.*, v1.cnt nb_intrsct_src, st_astext(v1.the_geom) as src_geom, v2.cnt as nb_intrsct_tgt, st_astext(v2.the_geom) as tgt_geom from public.test_agreg_lineaire t left join public.test_agreg_lineaire_vertices_pgr v1 on t.\"source\"=v1.id left join public.test_agreg_lineaire_vertices_pgr v2  on t.target=v2.id", c.connexionPsy)
nature_2_chaussees=['Route à 2 chaussées','Quasi-autoroute','Autoroute']
#donnees generale du jdd
df.set_index('id_ign', inplace=True)  # passer l'id_ign commme index ; necessaire sinon pb avec geometrie vu comme texte
df2_chaussees=df.loc[df.loc[:,'nature'].isin(nature_2_chaussees)] #isoler les troncon de voies decrits par 2 lignes

In [ ]:
toto=at.affecter_troncon_ligne('TRONROUT0000000109674477')

In [ ]:
toto

In [4]:
dico=at.affecter_troncon(df)
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

0eme occurence : TRONROUT0000000113489744 à 15:35:18 nb ligne traite : 0, nb ligne differente=0
300eme occurence : TRONROUT0000000129676524 à 15:35:27 nb ligne traite : 846, nb ligne differente=846
600eme occurence : TRONROUT0000000032825006 à 15:35:36 nb ligne traite : 1620, nb ligne differente=1620
900eme occurence : TRONROUT0000000032837043 à 15:35:37 nb ligne traite : 1749, nb ligne differente=1749
1200eme occurence : TRONROUT0000000032857215 à 15:35:41 nb ligne traite : 2046, nb ligne differente=2046
1500eme occurence : TRONROUT0000000032858991 à 15:35:45 nb ligne traite : 2458, nb ligne differente=2458
1800eme occurence : TRONROUT0000000032845694 à 15:35:49 nb ligne traite : 2969, nb ligne differente=2969
2100eme occurence : TRONROUT0000000032859654 à 15:35:53 nb ligne traite : 3348, nb ligne differente=3348
2400eme occurence : TRONROUT0000000033005488 à 15:35:57 nb ligne traite : 3894, nb ligne differente=3894
2700eme occurence : TRONROUT0000000113489812 à 15:36:00 nb ligne trai

In [ ]:
a